# Accessing  astronomical tables through the TAP with ADQL

The __[Table Access Protocol](http://www.ivoa.net/documents/TAP/)__ (TAP) protocol is under the hood of the Registry queries.  But it is in itself a powerful tool to search any VO table.  Here, we expand on its usage and that of the __[Astronomical Data Query Language](http://www.ivoa.net/documents/latest/ADQL.html)__ (ADQL) that it uses.  

* [1. Basic](#basic) Table Access Protocol queries
* [2. Cross-correlating](#cc) our own catalog with a HEASARC catalog
* [3. Combining](#combo) data from multiple catalogs and cross-correlating


In [1]:
## For debugging navo_utils
%load_ext autoreload
%autoreload 2

import numpy
## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

## For simple astropy tables
import astropy, io

## For handling ordinary astropy Tables
from astropy.table import Table

## For handling VO table type objects
from astropy.io import votable as apvot

## Use NAVO utility for Registry and Cone searches
import sys
sys.path.append("../aas_workshop_2018_summer/navo_utils")
from navo_utils.registry import Registry
from navo_utils.cone import Cone
from navo_utils.tap import Tap


<a id="basic"></a>

# 1. Basic Table Access Protocol queries

A TAP query is the most powerful way to search a catalog. A Simple Cone Search only allows you to ask for a position and radius, but TAP allows you to do much more.    

Supposed you already know that you want to query the "zcat". (More on discovering catalogs to query below.)  We use the registry to find the corresponding cone service and then submit our cone search.  

In [2]:
import astropy.coordinates as coord
coord=coord.SkyCoord.from_name("m51")
print(coord)


<SkyCoord (ICRS): (ra, dec) in deg
    (202.469575, 47.1952583)>


In [3]:
services=Registry.query(service_type='cone',source='zcat')
services

waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
str23,str9,str28,str24,str77,str57,str17,str10
gamma-ray#optical#x-ray,ROMABZCAT,ivo://nasa.heasarc/romabzcat,No Description Available,https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=romabzcat&amp;,https://heasarc.gsfc.nasa.gov/W3Browse/all/romabzcat.html,NASA/GSFC HEASARC,conesearch
optical,CFAZ,ivo://nasa.heasarc/zcat,No Description Available,https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=zcat&amp;,https://heasarc.gsfc.nasa.gov/W3Browse/all/zcat.html,NASA/GSFC HEASARC,conesearch
optical,ABELLZCAT,ivo://nasa.heasarc/abellzcat,No Description Available,https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=abellzcat&amp;,https://heasarc.gsfc.nasa.gov/W3Browse/all/abellzcat.html,NASA/GSFC HEASARC,conesearch


In [4]:
## Different tries seem to come back in different order!  Want CFAZ.  
table=Cone.query(service=services[1],coords=coord,radius=1)
table[0]

name,ra,dec,bmag,radial_velocity,radial_velocity_error,redshift,class,Search_Offset
,deg,deg,,km / s,km / s,,,
str10,float64,float64,float32,int32,int32,float64,int32,float64
13267+4631,202.20650,46.25850,15.10,0,0,--,9999,57.2374
N5173,202.10529,46.59082,14.12,2467,26,--,6300,39.2217
N5198,202.54745,46.66996,13.30,2569,25,--,6300,31.6790
N5169,202.04237,46.67154,14.70,2482,40,--,6200,35.9691
I4263,202.13883,46.92671,15.40,2663,25,--,6200,21.0327
N5194,202.46823,47.19815,9.03,474,23,--,6200,0.1819
N5195,202.49491,47.26792,10.94,558,23,--,6200,4.4802
1325+4754,201.87771,47.64124,--,18107,179,--,9999,35.9641


With the TAP, you can refine the search based on any other attribute in the given catalog.  

Now, we saw above with the results of the Registry query for cone services that the HEASARC lists every catalog as a separate cone service.  So we can find it with a search for source='heasarc%zcat', where the "%" is a wildcard.  But for TAP services, many services list in the Registry one TAP service that can access many catalogs.  We can find out what they are for a given service using the Tap.list_tables() function:

In [5]:
tap_services=Registry.query(service_type='table',source='heasarc')
url=str(tap_services[0]['access_url'])
print(url)

https://heasarc.gsfc.nasa.gov/xamin/vo/tap


In [6]:
Tap.list_tables(url)

Retrieving tables...
Parsing tables...
Done.
TAP_SCHEMA.TAP_SCHEMA.columns
TAP_SCHEMA.TAP_SCHEMA.key_columns
TAP_SCHEMA.TAP_SCHEMA.keys
TAP_SCHEMA.TAP_SCHEMA.schemas
TAP_SCHEMA.TAP_SCHEMA.tables
public.a1
public.a1point
public.a2lcpoint
public.a2lcscan
public.a2led
public.a2pic
public.a2point
public.a2rtraw
public.a2specback
public.a2spectra
public.a3
public.a4
public.a4spectra
public.aavsovsx
public.abell
public.abellzcat
public.acceptcat
public.acrs
public.actegsrcat
public.actssrcat
public.aegis20
public.aegis20id
public.aegisx
public.aegisxdcxo
public.agilecat
public.agileupvar
public.agnsdssxm2
public.agnsdssxmm
public.akaribsc
public.akaripsc
public.aknepdfcxo
public.alfperxmm
public.allwiseagn
public.ami10c15gz
public.amigps16gh
public.ansuvpscat
public.arcquincxo
public.ariel3a
public.ariel5
public.arxa
public.ascaegclus
public.ascagis
public.ascagps
public.ascalss
public.ascamaster
public.ascao
public.ascaprspec
public.ascasis
public.asiagosn
public.askapbeta
public.at20g
publ

Then after you've selected the *public.zcat* table, you want to know what columns you can query:  



In [7]:
Tap.list_columns(url,tablename="public.zcat")

<TableColumns names=('__row','name','ra','dec','lii','bii','bmag','radial_velocity','radial_velocity_error','ref_bmag','ref_radial_velocity','morph_type','bar_type','luminosity_class','structure','diameter_1','diameter_2','bt_mag','ugc_or_eso','distance','rfn_number','comments','redshift','ref_redshift','notes','class','__x_ra_dec','__y_ra_dec','__z_ra_dec')>

## Expressing queries in ADQL

So now you know all the possible column information in the zcat catalog, and you can do more than query on position (as in a cone search) but also on any other column.  The query has to be expressed in a language called __[ADQL](http://www.ivoa.net/documents/latest/ADQL.html)__.  

The basics of ADQL:

* *SELECT &#42; FROM my.interesting.catalog as cat...* 

says you want all ("&#42;") columns from the catalog called "my.interesting.catalog", which you will refer to in the rest of the query by the more compact name of "cat".  

Instead of returning all columns, you can 

* *SELECT cat.RA, cat.DEC, cat.bmag from catalog as cat...* 

to only return the columns you're interested in. To use multiple catalogs, your query could start, e.g.,

* *SELECT c1.RA,c1.DEC,c2.BMAG FROM catalog1 as c1 natural join catalog2 as c2...* 

says that you want to query two catalogs zipped together the "natural" way, i.e., by looking for a common column.

To select only some rows of the catalog based on the value in a column, you can add:  

* *WHERE cat.bmag < 14* 

says that you want to retrieve only those entries in the catalog whose bmag column has a value less than 14.

You can also append 

* *ORDER by cat.bmag* 

to return the result sorted ascending by one of the columns, adding *DESC* to the end for descending. 

A few special functions in the ADQL allow you to query regions:

* *WHERE contains( point('ICRS', cat.ra, cat.dec), circle('ICRS', 210.5, -6.5, 0.5))=1*

is how you would ask for any catalog entries whose RA,DEC lie within a circular region defined by RA,DEC 210.5,-6.5 and a radius of 0.5 (all in degrees).  The 'ICRS' specifies the coordinate system.  

See the ADQL documentation for more.

An ADQL query might look like this (we will discuss how to define the table and column names below):

In [8]:
##  Recall that in Python3, f"this is the value of my variable: {variable}" 
##  will print the value of the variable enclosed by the {}.  So this asks for 
##  rows of public.zcat where that row's ra and dec (cat.ra and cat.dec) 
##  are within radius 1deg of the given RA and DEC we got above for M51, and 
##  where the bmag column is less than 14.  
query=f"""SELECT ra, dec, Radial_Velocity, radial_velocity_error FROM public.zcat as cat where 
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',{coord.ra.deg},{coord.dec.deg},{1.0}))=1 and
    cat.bmag < 14
    order by cat.radial_velocity_error 
    """

In [9]:
results=Tap.query(url,query)
results

ra,dec,radial_velocity,radial_velocity_error
float64,float64,int32,int32
202.46823227,47.19814872,474,23
202.49490508,47.2679204,558,23
202.54744757,46.66995898,2569,25


(See the __[information on the zcat](https://heasarc.gsfc.nasa.gov/W3Browse/galaxy-catalog/zcat.html)__ for column information. We will use the 'radial_velocity' column rather than the 'redshift' column.)

<a id="cc"></a>
# 2. TAP:  Using the TAP to cross-correlate our objects with a catalog

Now to search all of our sources in one go, we need to upload our own table and do a 'cross-correlation' with the *zcat* table. For more on creating and working with VO tables, see that notebook.  Here, we just read one in that's already prepared:  

(These take a while, i.e. half a minute.)

In [10]:
query="""
    SELECT cat.ra, cat.dec, Radial_Velocity 
    FROM zcat cat, tap_upload.mysources mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,0.01))=1
    and Radial_Velocity > 0
    ORDER by cat.ra"""
url='https://heasarc.gsfc.nasa.gov/xamin_test/vo/tap'
zcattable=Tap.query(url,query,upload_file="data/my_sources.xml",upload_name='mysources')
zcattable

ra,dec,radial_velocity
float64,float64,int32
136.00074371,21.96791867,3093
146.70334308,22.01827217,7446
146.70334308,22.01827217,7597
148.77805631,14.29613296,7194
175.03940162,15.32725392,3325
191.54198643,30.73226569,6651
191.54739911,30.72338192,6517
194.91294475,28.89537435,6093
206.57163823,43.85050581,2229


FYI, this is what the same query looks like with the __[TapPlus library developed by ESDC](https://astroquery.readthedocs.io/en/latest/utils/tap.html)__.  A note about that library:  the "Plus" part refers to functions that are beyond the defined VO protocol and developed to work with extensions to the Gaia services.  Not all TapPlus functions will then work on all VO-compliant TAP services.  This function is generic:

In [11]:
#from astroquery.utils.tap.core import TapPlus
#xamin=TapPlus(url=tap_services[0]['access_url'])
#job=xamin.launch_job(query=query, upload_resource='data/my_sources.xml', upload_table_name="mysources", verbose=True)
#result = job.get_results()
#result.pprint()

<a id="combo"></a>

# 3.  Combining data from different catalogs and cross-correlating

## <font color=red>This needs a better science case to demonstrate.</font>

Now we'd like to take the redshift information (above, as a radial velocity) and determine a search radius to use for each galaxy based on its distance, so that we are are searching within a given physical distance. 

First, takeour table of objects and add an angDdeg column:

In [12]:
mytable = zcattable
## The radial_velocity is in km/s, and this is just c*z, so
c=3.0e5 # km/s
redshifts=mytable['radial_velocity'].filled(0.)/c  # Filling masked values with zero
mytable['redshift']=redshifts
from astropy import units
physdist=0.05*units.Mpc # 50 kpc physical distance

## This needs scipy.  
from astropy.cosmology import Planck15
angDdist=Planck15.angular_diameter_distance(mytable['redshift'])
## angDdist is returned from the astropy.cosmology module as a Quantity object, 
##  i.e. a value and a unit.  Arctan is smart enough not to operate on quantities
##  that aren't unitless.  So angDdist.value to just get the value.
angDrad=numpy.arctan(physdist/angDdist)
angDdeg=angDrad*units.deg/units.rad
mytable['angDdeg']=angDdeg
mytable

ra,dec,radial_velocity,redshift,angDdeg
,,,,deg
float64,float64,int32,float64,float64
136.00074371,21.96791867,3093,0.01031,0.0011097660332591386
146.70334308,22.01827217,7446,0.02482,0.0004691999680834195
146.70334308,22.01827217,7597,0.025323333333333333,0.0004601544041704519
148.77805631,14.29613296,7194,0.02398,0.00048514171179516753
175.03940162,15.32725392,3325,0.011083333333333334,0.001033309415720354
191.54198643,30.73226569,6651,0.02217,0.0005235991638639783
191.54739911,30.72338192,6517,0.021723333333333334,0.0005340756468748453
194.91294475,28.89537435,6093,0.02031,0.0005702614527842385


This time, rather than write the table to disk, we'll keep it in memory and give requests a "file-like" object using io.BytesIO():

In [13]:
## In memory only, use an IO stream. 
vot_obj=io.BytesIO()
print(mytable.columns)
apvot.writeto(apvot.from_table(mytable),vot_obj)
## (Reset the "file-like" object to the beginning.)
vot_obj.seek(0)
## 'uplt' is what we'll call it (for 'upload table') 
##   in the requests parameters below, or what you will:
files={'uplt':vot_obj}


<TableColumns names=('ra','dec','radial_velocity','redshift','angDdeg')>


Now we construct and run a query that uses the new angDdeg column in every row search.  This takes half a minute:

In [14]:
query="""SELECT cat.ra, cat.dec, cat.Radial_Velocity 
    FROM zcat cat, tap_upload.mytable mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,mt.angDdeg))=1
    and cat.Radial_Velocity > 0
    ORDER by cat.ra"""

mytable2=Tap.query(url,query,upload_file=vot_obj,upload_name='mytable')
mytable2

ra,dec,radial_velocity
float64,float64,int32
136.00074371,21.96791867,3093
146.70334308,22.01827217,7446
146.70334308,22.01827217,7446
146.70334308,22.01827217,7597
146.70334308,22.01827217,7597
148.77805631,14.29613296,7194
175.03940162,15.32725392,3325
191.54198643,30.73226569,6651
191.54739911,30.72338192,6517
